In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("KOSIS_API_KEY")


In [ ]:
import json
import re


def parse_kosis_loose(text: str):
    """
    KOSIS에서 내려오는 [{OBJ_ID:"A", ...}] 형태(키 따옴표 없음)를
    JSON으로 보정해서 list[dict]로 변환
    """
    t = text.strip()

    # {KEY: 또는 ,KEY: 를 {"KEY": 로 변환
    t = re.sub(r"([{,]\s*)([A-Za-z0-9_]+)\s*:", r'\1"\2":', t)

    # 혹시 값이 '...'로 오는 경우까지 대비(현재는 "..."라 그대로 OK)
    t = t.replace("'", '"')

    return json.loads(t)


In [ ]:
import pandas as pd
import requests as req

base_url = "https://kosis.kr/openapi/statisticsData.do"

params = {
    "method": "getMeta",
    "type": "ITM",
    "apiKey": key,
    "orgId": "101",
    "tblId": "DT_1YL20642",
    "format": "json",
}

res = req.get(base_url, params=params)

print(res.status_code)

res_text = res.text

print(res_text)

rows = parse_kosis_loose(res_text)
df = pd.DataFrame(rows)
df.to_csv("../../data/item_code.csv")

In [ ]:
import requests as req

url = "https://kosis.kr/openapi/Param/statisticsParameterData.do"
params = {
    "method": "getList",
    "apiKey": key,
    "orgId": "101",
    "tblId": "DT_1YL20642",
    "itmId": "T001+",
    "objL1": "32030",
    "objL2": "ALL",  # 필수
    "prdSe": "Y",
    "newEstPrdCnt": "3",
    "format": "json",
    "jsonVD": "Y",  # 이거 안넣으면 json 형태로 안옴
}

res = req.get(url, params=params)
print(res.status_code)
print(res.text[:300])


In [ ]:
import pandas as pd

df = pd.DataFrame(res.json())
df.head()